<a href="https://colab.research.google.com/github/Jencheng1/AWSSagemakerAutopilot/blob/main/Gemma_3_for_Video_Understanding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Gemma 3 For Video Understanding

Did you know that you can use Gemma 3 for video understanding?

Gemma 3 interleaving image and text allow for this. You can simply interleave timestamps and frames asking the model to summarize the events in the videos. Here's a small notebook to do so.

Install the release for Gemma 3 and login to access the model.

In [1]:
!pip install -q git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [6]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `test` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate whe

Let's load the model.

In [8]:
import torch
from transformers import AutoProcessor, Gemma3ForConditionalGeneration

ckpt = "google/gemma-3-4b-it"
model = Gemma3ForConditionalGeneration.from_pretrained(
    ckpt, device_map="auto", torch_dtype=torch.bfloat16,
)
processor = AutoProcessor.from_pretrained(ckpt)


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Download the video and downsample the frames from the video.

In [9]:
!wget https://huggingface.co/spaces/merve/llava-interleave/resolve/main/cats_1.mp4

--2025-03-13 16:34:03--  https://huggingface.co/spaces/merve/llava-interleave/resolve/main/cats_1.mp4
Resolving huggingface.co (huggingface.co)... 3.165.160.59, 3.165.160.11, 3.165.160.61, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.59|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/ec/8c/ec8c75d598fdb01954086dbec9c96aba9db39254c7754e1540ea550fe20caa1d/70db485f2feef1cabb7c63ede2f675c9b5bcce9fc610fe89f9abd90bbf0f0261?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27cats_1.mp4%3B+filename%3D%22cats_1.mp4%22%3B&response-content-type=video%2Fmp4&Expires=1741887243&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MTg4NzI0M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2VjLzhjL2VjOGM3NWQ1OThmZGIwMTk1NDA4NmRiZWM5Yzk2YWJhOWRiMzkyNTRjNzc1NGUxNTQwZWE1NTBmZTIwY2FhMWQvNzBkYjQ4NWYyZmVlZjFjYWJiN2M2M2VkZTJmNjc1YzliNWJjY2U5ZmM2MTBmZTg5ZjlhYmQ5MGJiZjBmM

In [10]:
import cv2
from PIL import Image
import numpy as np

def downsample_video(video_path):
    vidcap = cv2.VideoCapture(video_path)
    total_frames = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = vidcap.get(cv2.CAP_PROP_FPS)

    frames = []
    frame_indices = np.linspace(0, total_frames - 1, 10, dtype=int)

    for i in frame_indices:
        vidcap.set(cv2.CAP_PROP_POS_FRAMES, i)
        success, image = vidcap.read()
        if success:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Convert from BGR to RGB
            pil_image = Image.fromarray(image)
            timestamp = round(i / fps, 2)
            frames.append((pil_image, timestamp))

    vidcap.release()
    return frames


In [11]:
frames = downsample_video("cats_1.mp4")

In [ ]:
frames

[(<PIL.Image.Image image mode=RGB size=1920x1080>, 0.0),
 (<PIL.Image.Image image mode=RGB size=1920x1080>, 0.63),
 (<PIL.Image.Image image mode=RGB size=1920x1080>, 1.3),
 (<PIL.Image.Image image mode=RGB size=1920x1080>, 1.94),
 (<PIL.Image.Image image mode=RGB size=1920x1080>, 2.6),
 (<PIL.Image.Image image mode=RGB size=1920x1080>, 3.24),
 (<PIL.Image.Image image mode=RGB size=1920x1080>, 3.9),
 (<PIL.Image.Image image mode=RGB size=1920x1080>, 4.54),
 (<PIL.Image.Image image mode=RGB size=1920x1080>, 5.21),
 (<PIL.Image.Image image mode=RGB size=1920x1080>, 5.87)]

Here's our system prompt and the instruction. We will add frames and images on top of it.

In [12]:
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant."}]
    },

    {
        "role": "user",
        "content": [
            {"type": "text", "text": f"What is happening in this video? Summarize the events."}]
    }
]

In [13]:
messages[1]["content"][0]

{'type': 'text',
 'text': 'What is happening in this video? Summarize the events.'}

In [14]:
for frame in frames:
    image, timestamp = frame
    messages[1]["content"].append({"type": "text", "text": f"Frame {timestamp}:"})
    image.save(f"image_{timestamp}.png")
    messages[1]["content"].append({"type": "image", "url": f"image_{timestamp}.png"})

In [15]:
messages

[{'role': 'system',
  'content': [{'type': 'text', 'text': 'You are a helpful assistant.'}]},
 {'role': 'user',
  'content': [{'type': 'text',
    'text': 'What is happening in this video? Summarize the events.'},
   {'type': 'text', 'text': 'Frame 0.0:'},
   {'type': 'image', 'url': 'image_0.0.png'},
   {'type': 'text', 'text': 'Frame 0.63:'},
   {'type': 'image', 'url': 'image_0.63.png'},
   {'type': 'text', 'text': 'Frame 1.3:'},
   {'type': 'image', 'url': 'image_1.3.png'},
   {'type': 'text', 'text': 'Frame 1.94:'},
   {'type': 'image', 'url': 'image_1.94.png'},
   {'type': 'text', 'text': 'Frame 2.6:'},
   {'type': 'image', 'url': 'image_2.6.png'},
   {'type': 'text', 'text': 'Frame 3.24:'},
   {'type': 'image', 'url': 'image_3.24.png'},
   {'type': 'text', 'text': 'Frame 3.9:'},
   {'type': 'image', 'url': 'image_3.9.png'},
   {'type': 'text', 'text': 'Frame 4.54:'},
   {'type': 'image', 'url': 'image_4.54.png'},
   {'type': 'text', 'text': 'Frame 5.21:'},
   {'type': 'image', '

Preprocess our input and infer.

In [16]:
inputs = processor.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device)

In [17]:
input_len = inputs["input_ids"].shape[-1]

generation = model.generate(**inputs, max_new_tokens=500, do_sample=False)
generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)

OutOfMemoryError: CUDA out of memory. Tried to allocate 10.00 GiB. GPU 0 has a total capacity of 14.74 GiB of which 4.79 GiB is free. Process 7080 has 9.95 GiB memory in use. Of the allocated memory 9.81 GiB is allocated by PyTorch, and 20.32 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)